### Converting new bookings forecasts for deferred revenue model

Since Karen Burgess changed roles, I now have two sources of data for the bookings forecast

DX - 
DME - 

The data files they provide are very different and I need to change them to get one bookings dataframe


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import pickle


### DME bookings

In [2]:
filename = r'/Volumes/Treasury/Financial_Database/Deferred_Revenue/Inputs/DATA_2020_p12/DME_Bookings_FY21_Plan.xlsx'
sheetname = 'Raw'
df_DME = pd.read_excel(filename, sheetname)

In [3]:
df_DME.head(10)

,Metrics,Profit center,Market Area,Market Segement,GTM,Q1 2021,Q2 2021,Q3 2021,Q4 2021,2021
0,Total Subscription Attrition,EB10 - Creative,AMER (G),COMMERCIAL,Corporate,-2.855757e+06,-1.698423e+06,-2.570227e+06,-2.675469e+06,-9.799875e+06
1,Total Subscription Attrition,EB10 - Creative,AMER (G),EDUCATION,Corporate,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,-2.363023e+06
2,Total Subscription Attrition,EB10 - Creative,AMER (G),GOVERNMENT,Corporate,-7.577563e+05,-3.878279e+05,-1.658787e+06,-2.342317e+06,-5.146689e+06
3,Total Subscription Attrition,EB10 - Creative,AMER (G),NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04
4,Total Subscription Attrition,EB10 - Creative,North America (R),COMMERCIAL,Corporate,-2.800217e+06,-1.634119e+06,-2.537039e+06,-2.658157e+06,-9.629531e+06
5,Total Subscription Attrition,EB10 - Creative,North America (R),EDUCATION,Corporate,-2.251662e+05,-2.722325e+05,-1.283530e+06,-4.740660e+05,-2.254994e+06
6,Total Subscription Attrition,EB10 - Creative,North America (R),GOVERNMENT,Corporate,-7.356189e+05,-3.865962e+05,-1.658020e+06,-2.324543e+06,-5.104778e+06
7,Total Subscription Attrition,EB10 - Creative,North America (R),NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04
8,Total Subscription Attrition,EB10 - Creative,Canada (MA),COMMERCIAL,Corporate,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05
9,Total Subscription Attrition,EB10 - Creative,Canada (MA),EDUCATION,Corporate,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05


In [4]:
# Clean Column Names
df_DME.columns

Index(['Metrics', 'Profit center', 'Market Area', 'Market Segement', 'GTM',
       'Q1 2021', 'Q2 2021', 'Q3 2021', 'Q4 2021', '2021'],
      dtype='object')

In [5]:
df_DME = df_DME.rename(columns = {'Metrics': 'metrics',
                        'Profit center': 'profit_center',
                        'Market Area': 'market_area',
                        'Market Segement': 'segment',
                        'Q1 2021':'Q1_2021',
                        'Q2 2021':'Q2_2021',
                        'Q3 2021':'Q3_2021',
                        'Q4 2021':'Q4_2021' 
                        })

In [6]:
df_DME.columns

Index(['metrics', 'profit_center', 'market_area', 'segment', 'GTM', 'Q1_2021',
       'Q2_2021', 'Q3_2021', 'Q4_2021', '2021'],
      dtype='object')

In [7]:
df_DME['market_area'].value_counts()

United States (MA)           224
AMER (G)                     224
North America (R)            224
ASIA (G)                     180
Aus and New Zealand (MA)     172
ANZ (R)                      172
Southeast Asia (MA)          154
SEA (R)                      154
Korea (MA)                   132
Korea (R)                    132
Greater China (R)            130
EMEA (G)                     118
China (MA)                   116
Canada (MA)                  110
Northern Europe (R)          110
Central Europe (R)           109
Germany (MA)                 109
France (MA)                  108
India (R)                    108
Benelux (MA)                 108
India (MA)                   108
Southwest Europe (R)         108
United Kingdom (MA)          106
Switzerland (MA)             105
Iberica (MA)                 104
Italy (MA)                   104
Middle East (MA)             104
Nordic (MA)                  102
Hong Kong & Taiwan (MA)       98
SSA & Israel (MA)             88
Brazil (MA

In [8]:
df_DME.metrics.value_counts(dropna=False)

Net ACV                         1797
ASV & Usage Based               1510
Total Subscription Attrition    1096
Name: metrics, dtype: int64

## It looks like there are three seperate cuts of the data that are a hierarchy

1. G is GEO
2. R is region
3. MA is market Area

Going to split these into seperate sections and do successive fill forwards to cover all of them



In [9]:
# identify the characters in a string
df_DME['in_parens'] =  df_DME['market_area'].apply(lambda st: st[st.find("(")+1:st.find(")")])


In [10]:
df_DME['in_parens'].value_counts()

MA    2452
R     1379
G      572
Name: in_parens, dtype: int64

In [11]:
# now delete the parenthesis and the items between the parenthesis in the 'Market Area' column
df_DME.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4403 entries, 0 to 4402
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   metrics        4403 non-null   object 
 1   profit_center  4403 non-null   object 
 2   market_area    4403 non-null   object 
 3   segment        4403 non-null   object 
 4   GTM            4403 non-null   object 
 5   Q1_2021        4403 non-null   float64
 6   Q2_2021        4403 non-null   float64
 7   Q3_2021        4403 non-null   float64
 8   Q4_2021        4403 non-null   float64
 9   2021           4403 non-null   float64
 10  in_parens      4403 non-null   object 
dtypes: float64(5), object(6)
memory usage: 378.5+ KB


In [12]:
df_DME.head(20)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens
0,Total Subscription Attrition,EB10 - Creative,AMER (G),COMMERCIAL,Corporate,-2.855757e+06,-1.698423e+06,-2.570227e+06,-2.675469e+06,-9.799875e+06,G
1,Total Subscription Attrition,EB10 - Creative,AMER (G),EDUCATION,Corporate,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,-2.363023e+06,G
2,Total Subscription Attrition,EB10 - Creative,AMER (G),GOVERNMENT,Corporate,-7.577563e+05,-3.878279e+05,-1.658787e+06,-2.342317e+06,-5.146689e+06,G
3,Total Subscription Attrition,EB10 - Creative,AMER (G),NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,G
4,Total Subscription Attrition,EB10 - Creative,North America (R),COMMERCIAL,Corporate,-2.800217e+06,-1.634119e+06,-2.537039e+06,-2.658157e+06,-9.629531e+06,R
5,Total Subscription Attrition,EB10 - Creative,North America (R),EDUCATION,Corporate,-2.251662e+05,-2.722325e+05,-1.283530e+06,-4.740660e+05,-2.254994e+06,R
6,Total Subscription Attrition,EB10 - Creative,North America (R),GOVERNMENT,Corporate,-7.356189e+05,-3.865962e+05,-1.658020e+06,-2.324543e+06,-5.104778e+06,R
7,Total Subscription Attrition,EB10 - Creative,North America (R),NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,R
8,Total Subscription Attrition,EB10 - Creative,Canada (MA),COMMERCIAL,Corporate,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05,MA
9,Total Subscription Attrition,EB10 - Creative,Canada (MA),EDUCATION,Corporate,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05,MA


In [13]:
df_DME['market_area'] = df_DME['market_area'].apply(lambda st: st[0:st.find("(")-1])
df_DME.head(5)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens
0,Total Subscription Attrition,EB10 - Creative,AMER,COMMERCIAL,Corporate,-2.855757e+06,-1.698423e+06,-2.570227e+06,-2.675469e+06,-9.799875e+06,G
1,Total Subscription Attrition,EB10 - Creative,AMER,EDUCATION,Corporate,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,-2.363023e+06,G
2,Total Subscription Attrition,EB10 - Creative,AMER,GOVERNMENT,Corporate,-7.577563e+05,-3.878279e+05,-1.658787e+06,-2.342317e+06,-5.146689e+06,G
3,Total Subscription Attrition,EB10 - Creative,AMER,NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,G
4,Total Subscription Attrition,EB10 - Creative,North America,COMMERCIAL,Corporate,-2.800217e+06,-1.634119e+06,-2.537039e+06,-2.658157e+06,-9.629531e+06,R


In [14]:
df_DME['market_area'].value_counts()

Korea                   264
North America           224
AMER                    224
United States           224
India                   216
ASIA                    180
Aus and New Zealand     172
ANZ                     172
SEA                     154
Southeast Asia          154
Greater China           130
EMEA                    118
China                   116
Canada                  110
Northern Europe         110
Germany                 109
Central Europe          109
Southwest Europe        108
France                  108
Benelux                 108
United Kingdom          106
Switzerland             105
Middle East             104
Italy                   104
Iberica                 104
Nordic                  102
Japan                   100
Hong Kong & Taiwan       98
SSA & Israel             88
Brazil                   82
Latin America            82
Strat. Latin America     52
JPN                      50
Eastern Europe           48
Russia & CIS             44
Mexico              

In [15]:
df_DME.head(10)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens
0,Total Subscription Attrition,EB10 - Creative,AMER,COMMERCIAL,Corporate,-2.855757e+06,-1.698423e+06,-2.570227e+06,-2.675469e+06,-9.799875e+06,G
1,Total Subscription Attrition,EB10 - Creative,AMER,EDUCATION,Corporate,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,-2.363023e+06,G
2,Total Subscription Attrition,EB10 - Creative,AMER,GOVERNMENT,Corporate,-7.577563e+05,-3.878279e+05,-1.658787e+06,-2.342317e+06,-5.146689e+06,G
3,Total Subscription Attrition,EB10 - Creative,AMER,NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,G
4,Total Subscription Attrition,EB10 - Creative,North America,COMMERCIAL,Corporate,-2.800217e+06,-1.634119e+06,-2.537039e+06,-2.658157e+06,-9.629531e+06,R
5,Total Subscription Attrition,EB10 - Creative,North America,EDUCATION,Corporate,-2.251662e+05,-2.722325e+05,-1.283530e+06,-4.740660e+05,-2.254994e+06,R
6,Total Subscription Attrition,EB10 - Creative,North America,GOVERNMENT,Corporate,-7.356189e+05,-3.865962e+05,-1.658020e+06,-2.324543e+06,-5.104778e+06,R
7,Total Subscription Attrition,EB10 - Creative,North America,NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,R
8,Total Subscription Attrition,EB10 - Creative,Canada,COMMERCIAL,Corporate,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05,MA
9,Total Subscription Attrition,EB10 - Creative,Canada,EDUCATION,Corporate,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05,MA


In [16]:
df_DME['pc_ID'] = df_DME['profit_center'].apply(lambda st: st[0:st.find('-')])
df_DME['pc_descr'] = df_DME['profit_center'].apply(lambda st: st[st.find('-')+1:])

In [17]:
df_DME.sample(20)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens,pc_ID,pc_descr
936,Total Subscription Attrition,10800 - Acrobat Desk,Iberica,GOVERNMENT,Corporate,-1.129945e+03,-2.237286e+03,-1.103053e+03,-3.328039e+03,-7.798323e+03,MA,10800,Acrobat Desk
448,Total Subscription Attrition,IS18 - Stock Photography,Brazil,COMMERCIAL,Corporate,-2.966732e+03,0.000000e+00,-5.471778e+01,-1.772718e+02,-3.198722e+03,MA,IS18,Stock Photography
2288,Net ACV,GP15 - Document Cloud,Nordic,GOVERNMENT,Corporate,2.610646e+04,6.402820e+04,1.143170e+05,1.823834e+05,3.868350e+05,MA,GP15,Document Cloud
3809,ASV & Usage Based,GP15 - Document Cloud,ASIA,GOVERNMENT,Corporate,1.033750e+05,1.267500e+05,1.467500e+05,1.680000e+05,5.448750e+05,G,GP15,Document Cloud
3733,ASV & Usage Based,EB15 - Document Cloud,Southeast Asia,GOVERNMENT,Strategic,2.100000e+04,3.396644e+04,1.981394e+05,2.928341e+05,5.459399e+05,MA,EB15,Document Cloud
1589,Net ACV,10100 - Design,North America,EDUCATION,Corporate,-2.204490e+05,-2.710524e+05,-1.269479e+06,-4.704943e+05,-2.231475e+06,R,10100,Design
3363,ASV & Usage Based,10100 - Design,India,GOVERNMENT,Corporate,3.000000e+04,3.000000e+04,3.000000e+04,3.000000e+04,1.200000e+05,MA,10100,Design
4120,ASV & Usage Based,10800 - Acrobat Desk,Germany,GOVERNMENT,Corporate,9.707534e+05,3.212894e+05,4.400610e+05,7.628160e+05,2.494920e+06,MA,10800,Acrobat Desk
3012,ASV & Usage Based,EB10 - Creative,Southwest Europe,EDUCATION,Corporate,7.636001e+05,2.646463e+05,1.007810e+06,1.454195e+06,3.490251e+06,R,EB10,Creative
1943,Net ACV,13450 - Stock Photography,ASIA,COMMERCIAL,Corporate,2.000000e+04,2.500000e+04,2.500000e+04,3.000000e+04,1.000000e+05,G,13450,Stock Photography


adding columns geo, region, market_area (which already exists)

In [18]:
df_DME['geo'] = df_DME[df_DME['in_parens']=='G']['market_area']

In [19]:
df_DME.tail(60)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens,pc_ID,pc_descr,geo
4343,ASV & Usage Based,14400 - Adobe Sign,ANZ,EDUCATION,Strategic,4.091336e+04,5.561027e+04,4.691336e+04,5.557941e+04,1.990164e+05,R,14400,Adobe Sign,NaN
4344,ASV & Usage Based,14400 - Adobe Sign,ANZ,EDUCATION,Corporate,2.727558e+04,3.707351e+04,3.127558e+04,3.705294e+04,1.326776e+05,R,14400,Adobe Sign,NaN
4345,ASV & Usage Based,14400 - Adobe Sign,ANZ,GOVERNMENT,Strategic,6.018894e+04,6.518894e+04,8.467378e+04,8.585657e+04,2.959082e+05,R,14400,Adobe Sign,NaN
4346,ASV & Usage Based,14400 - Adobe Sign,Aus and New Zealand,COMMERCIAL,Strategic,1.379208e+05,1.407229e+05,2.215661e+05,2.098394e+05,7.100491e+05,MA,14400,Adobe Sign,NaN
4347,ASV & Usage Based,14400 - Adobe Sign,Aus and New Zealand,COMMERCIAL,Corporate,2.561386e+05,2.613425e+05,4.114798e+05,3.897018e+05,1.318663e+06,MA,14400,Adobe Sign,NaN
4348,ASV & Usage Based,14400 - Adobe Sign,Aus and New Zealand,EDUCATION,Strategic,4.091336e+04,5.561027e+04,4.691336e+04,5.557941e+04,1.990164e+05,MA,14400,Adobe Sign,NaN
4349,ASV & Usage Based,14400 - Adobe Sign,Aus and New Zealand,EDUCATION,Corporate,2.727558e+04,3.707351e+04,3.127558e+04,3.705294e+04,1.326776e+05,MA,14400,Adobe Sign,NaN
4350,ASV & Usage Based,14400 - Adobe Sign,Aus and New Zealand,GOVERNMENT,Strategic,6.018894e+04,6.518894e+04,8.467378e+04,8.585657e+04,2.959082e+05,MA,14400,Adobe Sign,NaN
4351,ASV & Usage Based,14400 - Adobe Sign,Greater China,COMMERCIAL,Strategic,3.136724e+04,2.757085e+04,4.308154e+04,2.925420e+04,1.312738e+05,R,14400,Adobe Sign,NaN
4352,ASV & Usage Based,14400 - Adobe Sign,Greater China,COMMERCIAL,Corporate,3.136724e+04,2.757085e+04,4.308154e+04,2.925420e+04,1.312738e+05,R,14400,Adobe Sign,NaN


In [20]:
df_DME['geo'] =df_DME['geo'].ffill()

In [21]:
df_DME['geo'].value_counts(dropna=False)

ASIA    1656
EMEA    1575
AMER    1022
JPN      150
Name: geo, dtype: int64

In [22]:
#df_DME.head(60)

### Comfortable that the forward fill of 'geo' worked well

### Working on the region now
Note: There will be a problem with the geo values here. They will all contain a region even if they are subtotals. This is OK because we will only be using the market area data and can recreate the region and geo data thanks to the new columns we have created


In [23]:
df_DME['region'] = df_DME[df_DME['in_parens']=='R']['market_area']
df_DME.head(20)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens,pc_ID,pc_descr,geo,region
0,Total Subscription Attrition,EB10 - Creative,AMER,COMMERCIAL,Corporate,-2.855757e+06,-1.698423e+06,-2.570227e+06,-2.675469e+06,-9.799875e+06,G,EB10,Creative,AMER,NaN
1,Total Subscription Attrition,EB10 - Creative,AMER,EDUCATION,Corporate,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,-2.363023e+06,G,EB10,Creative,AMER,NaN
2,Total Subscription Attrition,EB10 - Creative,AMER,GOVERNMENT,Corporate,-7.577563e+05,-3.878279e+05,-1.658787e+06,-2.342317e+06,-5.146689e+06,G,EB10,Creative,AMER,NaN
3,Total Subscription Attrition,EB10 - Creative,AMER,NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,G,EB10,Creative,AMER,NaN
4,Total Subscription Attrition,EB10 - Creative,North America,COMMERCIAL,Corporate,-2.800217e+06,-1.634119e+06,-2.537039e+06,-2.658157e+06,-9.629531e+06,R,EB10,Creative,AMER,North America
5,Total Subscription Attrition,EB10 - Creative,North America,EDUCATION,Corporate,-2.251662e+05,-2.722325e+05,-1.283530e+06,-4.740660e+05,-2.254994e+06,R,EB10,Creative,AMER,North America
6,Total Subscription Attrition,EB10 - Creative,North America,GOVERNMENT,Corporate,-7.356189e+05,-3.865962e+05,-1.658020e+06,-2.324543e+06,-5.104778e+06,R,EB10,Creative,AMER,North America
7,Total Subscription Attrition,EB10 - Creative,North America,NON-PROFIT,Corporate,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,-2.475830e+04,R,EB10,Creative,AMER,North America
8,Total Subscription Attrition,EB10 - Creative,Canada,COMMERCIAL,Corporate,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05,MA,EB10,Creative,AMER,NaN
9,Total Subscription Attrition,EB10 - Creative,Canada,EDUCATION,Corporate,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05,MA,EB10,Creative,AMER,NaN


In [24]:
df_DME['region'] = df_DME['region'].ffill()
df_DME['region'].value_counts(dropna=False)

Southwest Europe    598
North America       566
Northern Europe     510
SEA                 424
Latin America       418
Central Europe      415
ANZ                 344
Greater China       344
Japan               300
Korea               264
India               216
NaN                   4
Name: region, dtype: int64

In [25]:
df_DME.tail(40)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens,pc_ID,pc_descr,geo,region
4363,ASV & Usage Based,14400 - Adobe Sign,SEA,COMMERCIAL,Strategic,1.862253e+05,2.048968e+05,2.311119e+05,2.341869e+05,8.564209e+05,R,14400,Adobe Sign,ASIA,SEA
4364,ASV & Usage Based,14400 - Adobe Sign,SEA,COMMERCIAL,Corporate,7.981083e+04,8.781293e+04,9.904795e+04,1.003658e+05,3.670375e+05,R,14400,Adobe Sign,ASIA,SEA
4365,ASV & Usage Based,14400 - Adobe Sign,SEA,EDUCATION,Strategic,1.301613e+04,1.301613e+04,1.301613e+04,2.351613e+04,6.256452e+04,R,14400,Adobe Sign,ASIA,SEA
4366,ASV & Usage Based,14400 - Adobe Sign,SEA,EDUCATION,Corporate,5.578341e+03,5.578341e+03,5.578341e+03,1.007834e+04,2.681336e+04,R,14400,Adobe Sign,ASIA,SEA
4367,ASV & Usage Based,14400 - Adobe Sign,SEA,GOVERNMENT,Strategic,0.000000e+00,0.000000e+00,1.718894e+05,2.578341e+05,4.297235e+05,R,14400,Adobe Sign,ASIA,SEA
4368,ASV & Usage Based,14400 - Adobe Sign,Southeast Asia,COMMERCIAL,Strategic,1.862253e+05,2.048968e+05,2.311119e+05,2.341869e+05,8.564209e+05,MA,14400,Adobe Sign,ASIA,SEA
4369,ASV & Usage Based,14400 - Adobe Sign,Southeast Asia,COMMERCIAL,Corporate,7.981083e+04,8.781293e+04,9.904795e+04,1.003658e+05,3.670375e+05,MA,14400,Adobe Sign,ASIA,SEA
4370,ASV & Usage Based,14400 - Adobe Sign,Southeast Asia,EDUCATION,Strategic,1.301613e+04,1.301613e+04,1.301613e+04,2.351613e+04,6.256452e+04,MA,14400,Adobe Sign,ASIA,SEA
4371,ASV & Usage Based,14400 - Adobe Sign,Southeast Asia,EDUCATION,Corporate,5.578341e+03,5.578341e+03,5.578341e+03,1.007834e+04,2.681336e+04,MA,14400,Adobe Sign,ASIA,SEA
4372,ASV & Usage Based,14400 - Adobe Sign,Southeast Asia,GOVERNMENT,Strategic,0.000000e+00,0.000000e+00,1.718894e+05,2.578341e+05,4.297235e+05,MA,14400,Adobe Sign,ASIA,SEA


#### Now we only want to keep the market areas

In [26]:
df_DME = df_DME[df_DME['in_parens']=='MA'].copy()
df_DME['market_area'].value_counts(dropna=False)

United States           224
Aus and New Zealand     172
Southeast Asia          154
Korea                   132
China                   116
Canada                  110
Germany                 109
India                   108
France                  108
Benelux                 108
United Kingdom          106
Switzerland             105
Middle East             104
Iberica                 104
Italy                   104
Nordic                  102
Hong Kong & Taiwan       98
SSA & Israel             88
Brazil                   82
Strat. Latin America     52
Japan                    50
Eastern Europe           48
Russia & CIS             44
Mexico                   24
Name: market_area, dtype: int64

In [27]:
len(df_DME)

2452

In [28]:
df_DME.sample(20)

,metrics,profit_center,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,in_parens,pc_ID,pc_descr,geo,region
833,Total Subscription Attrition,IS15 - Acrobat DC,United Kingdom,GOVERNMENT,Corporate,-5.515244e+04,-1.114969e+05,-1.060158e+04,-8.917666e+03,-1.861686e+05,MA,IS15,Acrobat DC,EMEA,Northern Europe
125,Total Subscription Attrition,GP10 - Creative,Brazil,EDUCATION,Corporate,-1.743558e+04,-1.517062e+04,-5.544735e+03,-1.185218e+04,-5.000312e+04,MA,GP10,Creative,AMER,Latin America
3885,ASV & Usage Based,GP15 - Document Cloud,Nordic,COMMERCIAL,Corporate,1.037201e+06,1.213617e+06,8.755502e+05,2.835664e+06,5.962032e+06,MA,GP15,Document Cloud,EMEA,Northern Europe
931,Total Subscription Attrition,10800 - Acrobat Desk,Benelux,COMMERCIAL,Corporate,-7.467432e+03,-4.938996e+04,-3.167950e+04,-4.456739e+04,-1.331043e+05,MA,10800,Acrobat Desk,EMEA,Southwest Europe
877,Total Subscription Attrition,10800 - Acrobat Desk,Strat. Latin America,GOVERNMENT,Corporate,-1.813826e+03,0.000000e+00,0.000000e+00,0.000000e+00,-1.813826e+03,MA,10800,Acrobat Desk,AMER,Latin America
3209,ASV & Usage Based,IS10 - Creative - Professional,China,COMMERCIAL,Corporate,3.190000e+06,3.437500e+06,3.575000e+06,3.575000e+06,1.377750e+07,MA,IS10,Creative - Professional,ASIA,Greater China
796,Total Subscription Attrition,IS15 - Acrobat DC,Aus and New Zealand,GOVERNMENT,Strategic,-4.192217e+04,-9.617618e+04,-2.564620e+04,-5.044981e+04,-2.141944e+05,MA,IS15,Acrobat DC,ASIA,ANZ
1903,Net ACV,IS18 - Stock Photography,Italy,COMMERCIAL,Corporate,-1.802812e+04,1.888985e+04,5.523872e+03,1.853451e+04,2.492012e+04,MA,IS18,Stock Photography,EMEA,Southwest Europe
3128,ASV & Usage Based,GP10 - Creative,Eastern Europe,COMMERCIAL,Corporate,2.042941e+05,2.169124e+05,1.891615e+05,2.441975e+05,8.545656e+05,MA,GP10,Creative,EMEA,Central Europe
299,Total Subscription Attrition,IS10 - Creative - Professional,Middle East,EDUCATION,Corporate,-5.666875e+04,0.000000e+00,0.000000e+00,-2.578471e+04,-8.245346e+04,MA,IS10,Creative - Professional,EMEA,Northern Europe


In [29]:
df_DME = df_DME.drop(columns=['profit_center', 'in_parens' ])

In [30]:
df_DME.head()

,metrics,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,pc_ID,pc_descr,geo,region
8,Total Subscription Attrition,Canada,COMMERCIAL,Corporate,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05,EB10,Creative,AMER,North America
9,Total Subscription Attrition,Canada,EDUCATION,Corporate,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05,EB10,Creative,AMER,North America
10,Total Subscription Attrition,Canada,GOVERNMENT,Corporate,-1.073110e+05,-5.261821e+04,-4.332915e+04,-1.879336e+04,-2.220517e+05,EB10,Creative,AMER,North America
11,Total Subscription Attrition,United States,COMMERCIAL,Corporate,-2.763076e+06,-1.584732e+06,-2.420743e+06,-2.622828e+06,-9.391380e+06,EB10,Creative,AMER,North America
12,Total Subscription Attrition,United States,EDUCATION,Corporate,-2.251662e+05,-2.722325e+05,-1.167310e+06,-4.610124e+05,-2.125721e+06,EB10,Creative,AMER,North America


In [31]:
df_DME.columns

Index(['metrics', 'market_area', 'segment', 'GTM', 'Q1_2021', 'Q2_2021',
       'Q3_2021', 'Q4_2021', '2021', 'pc_ID', 'pc_descr', 'geo', 'region'],
      dtype='object')

In [32]:
df_DME['metrics'].value_counts()

Net ACV                         990
ASV & Usage Based               822
Total Subscription Attrition    640
Name: metrics, dtype: int64

##### We only want the 'Net ACV' bookings here, so we need to remove any other bookings type

In [33]:
df_DME = df_DME[df_DME['metrics']=='Net ACV']

In [34]:
df_DME.sample(10)

,metrics,market_area,segment,GTM,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021,pc_ID,pc_descr,geo,region
1276,Net ACV,United States,COMMERCIAL,Strategic,1.102294e+07,1.144855e+07,1.671312e+07,1.767977e+07,5.686438e+07,GP10,Creative,AMER,North America
1853,Net ACV,Aus and New Zealand,EDUCATION,Strategic,2.073888e+04,4.540631e+04,1.432653e+04,4.829066e+04,1.287624e+05,IS18,Stock Photography,ASIA,ANZ
2046,Net ACV,Mexico,COMMERCIAL,Corporate,-5.878793e+02,0.000000e+00,0.000000e+00,0.000000e+00,-5.878793e+02,EB15,Document Cloud,AMER,Latin America
1696,Net ACV,Germany,GOVERNMENT,Corporate,3.651596e+05,5.372816e+05,9.444298e+05,1.058117e+06,2.904988e+06,10100,Design,EMEA,Central Europe
1361,Net ACV,Southeast Asia,COMMERCIAL,Corporate,1.950000e+05,2.275000e+05,2.600000e+05,2.600000e+05,9.425000e+05,GP10,Creative,ASIA,SEA
1443,Net ACV,United States,NON-PROFIT,Corporate,-2.758205e+03,0.000000e+00,-1.490083e+04,-5.267702e+02,-1.818581e+04,IS10,Creative - Professional,AMER,North America
2151,Net ACV,France,GOVERNMENT,Corporate,2.923804e+04,6.166262e+04,5.099591e+04,1.221280e+05,2.640246e+05,EB15,Document Cloud,EMEA,Southwest Europe
2449,Net ACV,Benelux,GOVERNMENT,Corporate,1.049164e+05,1.300844e+05,1.946174e+05,1.961956e+05,6.258137e+05,IS15,Acrobat DC,EMEA,Southwest Europe
2604,Net ACV,Italy,GOVERNMENT,Corporate,1.032059e+04,8.071004e+03,1.597961e+04,1.945613e+04,5.382734e+04,10800,Acrobat Desk,EMEA,Southwest Europe
1836,Net ACV,United States,EDUCATION,Corporate,-4.717187e+03,-1.180052e+03,-1.405011e+04,-3.571672e+03,-2.351903e+04,IS18,Stock Photography,AMER,North America


In [35]:
df_DME = df_DME[['pc_descr', 'pc_ID', 'geo', 'region', 'market_area', 'segment', 'Q1_2021','Q2_2021', 'Q3_2021', 'Q4_2021', '2021']]

In [49]:
df_DME.head(10)

,pc_descr,pc_ID,geo,region,market_area,segment,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021
1110,Creative,EB10,AMER,North America,Canada,COMMERCIAL,6.095483e+05,6.234444e+05,9.189459e+05,9.843834e+05,3.136322e+06
1111,Creative,EB10,AMER,North America,Canada,COMMERCIAL,-3.714079e+04,-4.938637e+04,-1.162955e+05,-3.532906e+04,-2.381517e+05
1112,Creative,EB10,AMER,North America,Canada,EDUCATION,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,-1.292727e+05
1113,Creative,EB10,AMER,North America,Canada,GOVERNMENT,3.357112e+05,3.509312e+05,5.147951e+05,5.522203e+05,1.753658e+06
1114,Creative,EB10,AMER,North America,Canada,GOVERNMENT,-1.073110e+05,-5.261821e+04,-4.332915e+04,-1.879336e+04,-2.220517e+05
1115,Creative,EB10,AMER,North America,United States,COMMERCIAL,1.102294e+07,1.144855e+07,1.671312e+07,1.767977e+07,5.686438e+07
1116,Creative,EB10,AMER,North America,United States,COMMERCIAL,-3.276029e+05,9.436072e+05,1.282755e+06,1.322818e+06,3.221577e+06
1117,Creative,EB10,AMER,North America,United States,EDUCATION,3.921053e+06,3.845633e+06,5.875375e+06,6.629117e+06,2.027118e+07
1118,Creative,EB10,AMER,North America,United States,EDUCATION,-2.251662e+05,-2.722325e+05,-1.167310e+06,-4.610124e+05,-2.125721e+06
1119,Creative,EB10,AMER,North America,United States,GOVERNMENT,1.944009e+06,1.933252e+06,2.929489e+06,3.274859e+06,1.008161e+07


In [36]:
df_DME.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990 entries, 1110 to 2892
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pc_descr     990 non-null    object 
 1   pc_ID        990 non-null    object 
 2   geo          990 non-null    object 
 3   region       990 non-null    object 
 4   market_area  990 non-null    object 
 5   segment      990 non-null    object 
 6   Q1_2021      990 non-null    float64
 7   Q2_2021      990 non-null    float64
 8   Q3_2021      990 non-null    float64
 9   Q4_2021      990 non-null    float64
 10  2021         990 non-null    float64
dtypes: float64(5), object(6)
memory usage: 92.8+ KB


### The DX bookings have no information about segment. We need to delete segment and then group by to following
- pc_descr
- geo
- region
- market_area


In [62]:
df_DME = df_DME.drop(columns = 'segment')
df_DME.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990 entries, 1110 to 2892
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pc_descr     990 non-null    object 
 1   pc_ID        990 non-null    object 
 2   geo          990 non-null    object 
 3   region       990 non-null    object 
 4   market_area  990 non-null    object 
 5   Q1_2021      990 non-null    float64
 6   Q2_2021      990 non-null    float64
 7   Q3_2021      990 non-null    float64
 8   Q4_2021      990 non-null    float64
 9   2021         990 non-null    float64
dtypes: float64(5), object(5)
memory usage: 85.1+ KB


In [64]:
df_DME.sum()

pc_descr        Creative Creative Creative Creative Creative ...
pc_ID          EB10 EB10 EB10 EB10 EB10 EB10 EB10 EB10 EB10 E...
geo            AMERAMERAMERAMERAMERAMERAMERAMERAMERAMERAMERAM...
region         North AmericaNorth AmericaNorth AmericaNorth A...
market_area    CanadaCanadaCanadaCanadaCanadaUnited StatesUni...
Q1_2021                                              3.35128e+08
Q2_2021                                              4.02485e+08
Q3_2021                                              4.98973e+08
Q4_2021                                               6.7508e+08
2021                                                 1.91167e+09
dtype: object

In [70]:
df_DME = df_DME.groupby(by = ['pc_descr', 'geo', 'region', 'market_area']).sum()
df_DME.sum()

Q1_2021    3.351281e+08
Q2_2021    4.024845e+08
Q3_2021    4.989733e+08
Q4_2021    6.750802e+08
2021       1.911666e+09
dtype: float64

In [71]:
df_DME.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 227 entries, (' Acrobat DC', 'AMER', 'Latin America', 'Brazil') to (' Stock Photography', 'JPN', 'Japan', 'Japan')
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Q1_2021  227 non-null    float64
 1   Q2_2021  227 non-null    float64
 2   Q3_2021  227 non-null    float64
 3   Q4_2021  227 non-null    float64
 4   2021     227 non-null    float64
dtypes: float64(5)
memory usage: 10.4+ KB


In [76]:
df_DME.head(10)

,pc_descr,geo,region,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021,2021
0,Acrobat DC,AMER,Latin America,Brazil,1.617778e+05,1.777746e+05,2.596567e+05,3.974719e+05,9.966810e+05
1,Acrobat DC,AMER,Latin America,Mexico,-5.878793e+02,0.000000e+00,0.000000e+00,0.000000e+00,-5.878793e+02
2,Acrobat DC,AMER,Latin America,Strat. Latin America,-4.793599e+03,-1.201923e+03,-2.246623e+01,0.000000e+00,-6.017988e+03
3,Acrobat DC,AMER,North America,Canada,7.286022e+05,8.484676e+05,1.233965e+06,1.953663e+06,4.764697e+06
4,Acrobat DC,AMER,North America,United States,9.789297e+06,1.080222e+07,1.504463e+07,2.315904e+07,5.879518e+07
5,Acrobat DC,ASIA,ANZ,Aus and New Zealand,1.006561e+06,1.162570e+06,1.295138e+06,1.698991e+06,5.163260e+06
6,Acrobat DC,ASIA,Greater China,China,1.085210e+06,1.131169e+06,1.128201e+06,1.329854e+06,4.674434e+06
7,Acrobat DC,ASIA,Greater China,Hong Kong & Taiwan,1.573385e+05,1.648655e+05,1.902880e+05,1.894954e+05,7.019874e+05
8,Acrobat DC,ASIA,India,India,4.941753e+05,4.768403e+05,6.236849e+05,6.620033e+05,2.256704e+06
9,Acrobat DC,ASIA,Korea,Korea,1.649261e+05,2.844877e+05,3.649871e+05,3.934598e+05,1.207861e+06


In [75]:
df_DME.reset_index(inplace=True)


### We need to add a "BU" and "segment" field to these
The BU will be Digital Media and the segment will all be lumped into Creative


In [77]:
df_DME['BU'] = 'Digital Media'
df_DME['segment'] = 'Creative'

In [ ]:
df_DME.head

In [37]:
df_DME.pc_descr.value_counts(dropna=False)

 Creative                   184
 Document Cloud             170
 Stock Photography          104
 Adobe Sign                 102
 Design                      92
 Creative - Professional     92
 Acrobat DC                  81
 Acrobat Desk                81
 DCE                         43
 CCE + Stock                 41
Name: pc_descr, dtype: int64

## Done with DME now moving to DX



In [51]:
filename = r'/Volumes/Treasury/Financial_Database/Deferred_Revenue/Inputs/DATA_2020_p12/DX_Bookings_FY21_Plan.xlsx'
sheetname = 'Sheet1'
start=12
df_DX = pd.read_excel(filename, sheetname, skiprows=start)

In [52]:
df_DX.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,2021,Q1 2021,Q2 2021,Q3 2021,Q4 2021
0,Enterprise,All Market Areas,12400 - Adobe Analytics,7.223474e+07,1.316699e+07,1.652153e+07,1.814789e+07,2.439833e+07
1,Enterprise,All Market Areas,14800 - Customer Journey Analytics,2.861741e+07,5.059796e+06,6.665272e+06,6.952678e+06,9.939666e+06
2,Enterprise,All Market Areas,12610 - Audience Manager,1.028093e+07,1.761821e+06,2.332947e+06,2.492824e+06,3.693333e+06
3,Enterprise,All Market Areas,14700 - Real Time CDP,3.093418e+07,5.656254e+06,6.457362e+06,7.532753e+06,1.128781e+07
4,Enterprise,All Market Areas,12500 - Adobe Target,4.010254e+07,7.320283e+06,9.058720e+06,9.848370e+06,1.387517e+07
5,Enterprise,All Market Areas,14600 - Adobe Exp Platform,5.381726e+07,9.536363e+06,1.243553e+07,1.317641e+07,1.866895e+07
6,Enterprise,All Market Areas,13010 - Assets,7.243556e+07,1.310389e+07,1.560123e+07,1.810836e+07,2.562207e+07
7,Enterprise,All Market Areas,13020 - Sites,1.803132e+08,3.175266e+07,4.029789e+07,4.440377e+07,6.385891e+07
8,Enterprise,All Market Areas,13030 - Forms,3.738079e+07,6.195738e+06,8.025811e+06,9.466740e+06,1.369250e+07
9,Enterprise,All Market Areas,12630 - Magento,6.785980e+07,1.224868e+07,1.467157e+07,1.674743e+07,2.419212e+07


In [53]:
df_DX = df_DX.rename(columns = {'Unnamed: 0': 'segment',
                                'Unnamed: 1': 'market_area',
                                'Unnamed: 2': 'profit_center',
                                'Q1 2021':'Q1_2021',
                                'Q2 2021':'Q2_2021',
                                'Q3 2021':'Q3_2021',
                                'Q4 2021':'Q4_2021' 
                                })

## It looks like there are three seperate cuts of the data that are a hierarchy again, but they also have an 'All Market Areas' that is a total by profit center. We don't need this but there is no need to change the code here.
It will be removed when we get to the filter on region

1. G is GEO
2. R is region
3. MA is market Area

Going to split these into seperate sections and do successive fill forwards to cover all of them



In [54]:
df_DX.head()

,segment,market_area,profit_center,2021,Q1_2021,Q2_2021,Q3_2021,Q4_2021
0,Enterprise,All Market Areas,12400 - Adobe Analytics,7.223474e+07,1.316699e+07,1.652153e+07,1.814789e+07,2.439833e+07
1,Enterprise,All Market Areas,14800 - Customer Journey Analytics,2.861741e+07,5.059796e+06,6.665272e+06,6.952678e+06,9.939666e+06
2,Enterprise,All Market Areas,12610 - Audience Manager,1.028093e+07,1.761821e+06,2.332947e+06,2.492824e+06,3.693333e+06
3,Enterprise,All Market Areas,14700 - Real Time CDP,3.093418e+07,5.656254e+06,6.457362e+06,7.532753e+06,1.128781e+07
4,Enterprise,All Market Areas,12500 - Adobe Target,4.010254e+07,7.320283e+06,9.058720e+06,9.848370e+06,1.387517e+07


In [55]:
# identify the characters in a string
df_DX['in_parens'] =  df_DX['market_area'].apply(lambda st: st[st.find("(")+1:st.find(")")])

In [56]:
df_DX['market_area'] = df_DX['market_area'].apply(lambda st: st[0:st.find("(")-1])

In [57]:
df_DX['pc_ID'] = df_DX['profit_center'].apply(lambda st: st[0:st.find('-')])
df_DX['pc_descr'] = df_DX['profit_center'].apply(lambda st: st[st.find('-')+1:])

In [58]:
df_DX['geo'] = df_DX[df_DX['in_parens']=='G']['market_area']
df_DX['geo'] =df_DX['geo'].ffill()

In [59]:
df_DX['region'] = df_DX[df_DX['in_parens']=='R']['market_area']
df_DX['region'] = df_DX['region'].ffill()

In [60]:
# filter to just include market area
df_DX = df_DX[df_DX['in_parens']=='MA'].copy()

In [50]:
# drop unnecessary columns and reorder the columns
df_DX = df_DX.drop(columns=['profit_center', 'in_parens' ])
df_DX = df_DX[['pc_descr', 'pc_ID', 'geo', 'region', 'market_area', 'segment', 'Q1_2021','Q2_2021', 'Q3_2021', 'Q4_2021', '2021']]

KeyError: "['profit_center' 'in_parens'] not found in axis"

In [61]:
df_DX.head(20)

,segment,market_area,profit_center,2021,Q1_2021,Q2_2021,Q3_2021,Q4_2021,in_parens,pc_ID,pc_descr,geo,region
43,Enterprise,Canada,12400 - Adobe Analytics,3.211144e+06,5.941324e+05,8.360044e+05,8.014237e+05,9.795836e+05,MA,12400,Adobe Analytics,AMER,North America
44,Enterprise,Canada,14800 - Customer Journey Analytics,4.920000e+05,9.103083e+04,1.280896e+05,1.227913e+05,1.500883e+05,MA,14800,Customer Journey Analytics,AMER,North America
45,Enterprise,Canada,12610 - Audience Manager,4.982563e+05,9.218839e+04,1.297184e+05,1.243527e+05,1.519968e+05,MA,12610,Audience Manager,AMER,North America
46,Enterprise,Canada,14700 - Real Time CDP,8.007127e+05,1.481495e+05,2.084613e+05,1.998385e+05,2.442634e+05,MA,14700,Real Time CDP,AMER,North America
47,Enterprise,Canada,12500 - Adobe Target,1.437567e+06,2.659815e+05,3.742629e+05,3.587818e+05,4.385405e+05,MA,12500,Adobe Target,AMER,North America
48,Enterprise,Canada,14600 - Adobe Exp Platform,1.455000e+06,2.692070e+05,3.788016e+05,3.631327e+05,4.438587e+05,MA,14600,Adobe Exp Platform,AMER,North America
49,Enterprise,Canada,13010 - Assets,1.870699e+06,3.461205e+05,4.870266e+05,4.668811e+05,5.706707e+05,MA,13010,Assets,AMER,North America
50,Enterprise,Canada,13020 - Sites,3.426939e+06,6.340593e+05,8.921856e+05,8.552810e+05,1.045414e+06,MA,13020,Sites,AMER,North America
51,Enterprise,Canada,13030 - Forms,7.224152e+05,1.336627e+05,1.880770e+05,1.802973e+05,2.203782e+05,MA,13030,Forms,AMER,North America
52,Enterprise,Canada,12630 - Magento,2.209882e+06,4.088768e+05,5.753312e+05,5.515330e+05,6.741411e+05,MA,12630,Magento,AMER,North America
